In [1]:
###############################################
#                                             #
#                  READ ME                    #
#                                             #
###############################################
#
# This file is broken into three parts.
#
#   1. Importing necessary libraries and data
#   2. The training and testing of a new model
#   3. Loading in a previously trained model to test on some data
#
# In order to run parts 2 or 3, you will need to run part 1
#
# Any questions or issues email me:
#       robe1157@msu.edu
#

In [2]:
###############################################
#                                             #
#                  PART ONE                   #
#                                             #
###############################################

In [3]:
# Import Basic libraries

import numpy as np
import pandas as pd
import logging
import anndata as ad
import pickle

In [4]:
# Other libraries that are required to run

logging.basicConfig(level=logging.INFO)

from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt

import logging
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from scipy.sparse import csc_matrix
import magic

from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import scale

In [5]:
#SET YOUR NEEDED PATHS

logging.basicConfig(level=logging.INFO)

# YOU ONLY NEED THESE TWO LINES IF TRAINING A NEW MODEL
train_gex = ad.read_h5ad("Gex_processed_training.h5ad") #gex is gene expression which are RNA; Training data input
train_adt = ad.read_h5ad("Adt_processed_training.h5ad") # adt is protein; Training data response


# IF ALL YOU NEED IS TO LOAD TESTING DATA THEN USE THESE LINE AND COMMENT THE OTHERS OUT
test_gex = ad.read_h5ad("Gex_processed_testing.h5ad") #gex is gene expression which are RNA; Training data input
test_adt = ad.read_h5ad("Adt_processed_testing.h5ad") # adt is protein; Training data response

In [6]:
# This will get passed to the method
input_train_gex = train_gex
input_train_adt = train_adt
input_test_gex =  test_gex

# This will get passed to the metric
true_test_adt =  test_adt

In [7]:
def calculate_rmse(true_test_adt, pred_test_adt):
    return  mean_squared_error(true_test_adt.X.toarray(), pred_test_adt.X, squared = False)

In [8]:
###############################################
#                                             #
#                  PART TWO                   #
#                                             #
###############################################

# Do not run if you aren't looking to train a model

In [99]:
# Do PCA on the input data
'''Baseline method training a linear regressor on the input data'''
input_gex = ad.concat(
    {"train": input_train_gex, "test": input_test_gex},
    axis = 0,
    join = "outer",
    label = "group",
    fill_value = 0,
    index_unique = "-", 
)

# Do PCA on the input data
n = 50
logging.info('Performing dimensionality reduction on GEX values...')
embedder_gex = TruncatedSVD(n_components=n)
gex_pca = embedder_gex.fit_transform(input_gex.X)

INFO:root:Performing dimensionality reduction on GEX values...


In [59]:
with open('pca_model.pkl', 'wb') as f:
    pickle.dump(embedder_gex, f)


In [ ]:
def baseline_linear(input_train_gex, input_train_adt, input_test_gex):
    '''Baseline method training a linear regressor on the input data'''
    input_gex = ad.concat(
        {"train": input_train_gex, "test": input_test_gex},
        axis = 0,
        join = "outer",
        label = "group",
        fill_value = 0,
        index_unique = "-", 
    )
    
    # Do PCA on the input data
    n = 50
    logging.info('Performing dimensionality reduction on GEX values...')
    embedder_gex = TruncatedSVD(n_components=n)
    gex_pca = embedder_gex.fit_transform(input_gex.X)
        
    
    # split dimension reduction GEX back up for training
    X_train = gex_pca[input_gex.obs['group'] == 'train']
    X_test = gex_pca[input_gex.obs['group'] == 'test']
    y_train = input_train_adt.X.toarray()

    assert len(X_train) + len(X_test) == len(gex_pca)
    
    logging.info('Running Linear regression...')
    
    reg = Ridge()
    
    # Train the model on the PCA reduced gex 1 and 2 data   
    # Hyperparameters for our network
    input_size = n
    hidden_sizes = [5000,500,100]
    output_size = 134

    # Build a feed-forward network
    model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                          nn.BatchNorm1d(hidden_sizes[0]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                          nn.BatchNorm1d(hidden_sizes[1]),
			  nn.ReLU(),
			  nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                          nn.BatchNorm1d(hidden_sizes[2]),
                          nn.Sigmoid(),
                          nn.Linear(hidden_sizes[2], output_size))
    print(model)

    print("Building tensor objects")

    X_train_tensor = torch.Tensor(X_train)
    y_train_tensor = torch.Tensor(y_train)
    X_test_tensor  = torch.Tensor(X_test)
    #y_test_tensor  = torch.Tensor(y_test)

    mydata_train = TensorDataset(X_train_tensor,y_train_tensor)
    #mydata_test  = TensorDataset(X_test_tensor,y_test_tensor)

    trainloader = torch.utils.data.DataLoader(mydata_train, batch_size=20, shuffle=True, drop_last=True)
    #testloader = torch.utils.data.DataLoader(mydata_test, batch_size=10, shuffle=True, drop_last=True)

    criterion = nn.MSELoss()# Optimizers require the parameters to optimize and a learning rate
    optimizer = optim.SGD(model.parameters(), lr=0.003)
    epochs = 15
 
    print("Running epochs")

    for e in range(epochs):
        running_loss = 0
        for data, target in trainloader:

            # Training pass
            optimizer.zero_grad()

            output = model(data) #<--- note this line is using the model you set up at the beginning of this section
            output = output.float()
            target = target.float()
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        else:
            print(f"Training loss: {running_loss/len(trainloader)}")
    
    predict = model(X_test_tensor)
    y_pred = predict.detach().numpy()

 
    # Project the predictions back to the adt feature space
    
    pred_test_adt = ad.AnnData(
        X = y_pred,
        obs = input_test_gex.obs,
        var = input_train_adt.var,
    
    )
    
    # Add the name of the method to the result
    pred_test_adt.uns["method"] = "linear"
    
    return pred_test_adt

# Run prediction
pred_test_adt = baseline_linear(input_train_gex, input_train_adt, input_test_gex)
# Calculate RMSE
rmse = calculate_rmse(true_test_adt, pred_test_adt)
# Print results
print(f'{pred_test_adt.uns["method"]} had a RMSE of {rmse:.4f}')


In [9]:
###############################################
#                                             #
#                PART THREE                   #
#                                             #
###############################################

# Here we will be loading in PCA and Pytorch models then run the model on the new data

In [10]:
# specify the path to the saved model
PATH = "mlp_01/model_01.pth"
# specify the path to the PCA model
PCA_PATH = 'mlp_01/pca_model.pkl'

In [12]:
# Build a new instance of the model 
# Hyperparameters for our network
input_size = 50
hidden_sizes = [5000,5000,5000]
output_size = 134

# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                        nn.BatchNorm1d(hidden_sizes[0]),
                        nn.ReLU(),
                        nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                        nn.BatchNorm1d(hidden_sizes[1]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                        nn.BatchNorm1d(hidden_sizes[2]),
                        nn.Tanh(),
                        nn.Linear(hidden_sizes[2], output_size))
print(model)

Sequential(
  (0): Linear(in_features=50, out_features=5000, bias=True)
  (1): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=5000, out_features=5000, bias=True)
  (4): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Linear(in_features=5000, out_features=5000, bias=True)
  (7): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Tanh()
  (9): Linear(in_features=5000, out_features=134, bias=True)
)


In [13]:
# LOAD IN MODEL
model.load_state_dict(torch.load(PATH))
model.eval()

# LOAD IN PCA
with open(PCA_PATH, 'rb') as f:
    pca = pickle.load(f)

In [14]:
# Fit the testing data
X_test = pca.transform(input_test_gex.X)

In [15]:
# Run the model on the fitted data
X_test_tensor  = torch.Tensor(X_test)

predict = model(X_test_tensor)
y_pred = predict.detach().numpy()

In [16]:
# Calculate RMSE
mean_squared_error(true_test_adt.X.toarray(), y_pred, squared = False)

0.3591859631554783